checklist:


Drops
- address1 drop
- exception: clean
- new_value: dropped it (doesn't contain anything)
- current_value: dropped it (doesn't contain anything
- tax_type: clean (create a data dictionary for this column) 
- instrument_no
- sub_neighborhood': A, B, C...huh? 
- 'tax_class: all the same (residential)
- homestead: I dummied this
- building_area: nulls
- triennial_group: no cleaning necessary (float)
- address: I parsed this
- instrument_no: no clue
- tax_class2: 1 for residential
- building_area: empty (for now. check again once you get the full dataset)

cleaned
- assessor: cleaned (titlecase)
- tax_class2: cleaned and filtered
- use_code: clean (create a data dictionary for this column)
- neighborhood: clean
- owner_name: cleaned (converted to title case)
- address: cleaned (spend an hour on this!
- sale_price: cleaned (dropped commas and dollar signs)
- land_area: clean (dropped commas)
- ward: float status. cleaning not needed
- land_2017: cleaned (dropped commas and dollar signs)
- land_2018: cleaned (dropped commas and dollar signs)
- improvements_2017: cleaned (dropped commas and dollar signs)
- improvements_2018: cleaned (dropped commas and dollar signs)
- value_2017: cleaned (dropped commas and dollar signs)
- value_2018: cleaned (dropped commas and dollar signs)
- assessment_2017: cleaned (dropped commas and dollar signs)
- assessment_2018: cleaned (dropped commas and dollar signs)

filters:
- drop duplicates
- tax_class2: cleaned and filtered
- tax_class2 = 1
- sale_price > 100
- zip_code: need to filter to ^2\d+ zip codes
- city: non-dc values (look at city column)
- code: 1, only residential

Columns created
- homestead: dummified and column cleaned [drop for arima]
- zip_code: created [need to filter]
- address_1 column
- qtr
- month
- year

I need to:
- consider dropping land_area < '4'
- look at home values that are \$1
- ### add lats and longs
- eda (sns plot)
- search for 'Not Available'

I can if I have time:
- use_code: create a data dictionary for this column) [super important]
- tax_class2: create a data dictionary for this column)
- look into subneighborhood: #wtf is this?


In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
path = 'otr copy 7.csv'
df = pd.read_csv(path, parse_dates=['date'], infer_datetime_format=True)

In [3]:
#df = df.address.notnull() #http://bit.ly/2zk56jD

In [4]:
df.replace('', np.nan, inplace=True) #I KNOW that there are fucking empty cells. gotta fill them in.

In [5]:
df.dropna(inplace=True)

In [6]:
df['neighborhood'] = df['neighborhood'].str.title()
df['neighborhood'] = df['neighborhood'].str.replace('American Univ. Park','American University Park')
df['neighborhood'] = df['neighborhood'].str.replace('N. Cleveland Park','North Cleveland Park')

In [7]:
df['neighborhood'].unique() #this is not exhustive, but at least it's clean now

array(['American University Park', 'Anacostia', 'Barry Farms', 'Berkley',
       'Bolling Afb & Naval Res', 'Brentwood', 'Brightwood', 'Brookland',
       'Burleith', 'Capitol Hill', 'Central', 'Chevy Chase', 'Chillum',
       'Cleveland Park', 'Colonial Village', 'Columbia Heights',
       'Congress Heights', 'Crestwood', 'D.C. Village', 'Deanwood',
       'Eckington', 'Foggy Bottom', 'Forest Hills', 'Fort Dupont Park',
       'Fort Lincoln', 'Foxhall', 'Garfield', 'Georgetown', 'Glover Park',
       'Hawthorne', 'Hillcrest', 'Kalorama', 'Kent', 'Ledroit Park',
       'Lily Ponds', 'Marshall Heights', 'Mass. Ave. Heights',
       'Michigan Park', 'Mount Pleasant', 'N. Anacostia Park',
       'North Cleveland Park', 'Observatory Circle', 'Old City I',
       'Old City Ii', 'Palisades', 'Petworth', 'R.L.A. Ne', 'R.L.A. Sw',
       'Randle Heights', 'Riggs Park', 'Shepherd Park', '16Th St. Heights',
       'S. Anacostia Park', 'Spring Valley'], dtype=object)

In [8]:
df['use_code2'] = df['use_code'].str.extract('(^\d{1,})', expand = True)

In [9]:
df['use_code2']=df['use_code2'].str.strip()

In [10]:
df.drop(['use_code'], 1, inplace = True)

In [11]:
df['tax_type'] = df['tax_type'].str.replace(u'\xa0', u' ')
#\xa0 is  non-breaking space in Latin1 (ISO 8859-1). replace with a space

In [12]:
df['tax_type2'] = df['tax_type'].str.extract('(^\w{2})', expand = True)

In [13]:
df.drop(['tax_type'], 1, inplace = True)

In [14]:
df['tax_type2']=df['tax_type2'].str.strip()

In [15]:
df['tax_class'] = df['tax_class'].str.replace(u'\xa0', u' ')

In [16]:
df['tax_class'].unique()

array(['001 - Residential              ',
       '002 - Commercial               ',
       '003 - Vacant                   ',
       '004 - Blighted                 ',
       '000 - Non-Taxable              ', ' '], dtype=object)

In [17]:
df['tax_class2'] = df['tax_class'].str.extract(r'([0-9]\b)', expand = True)

In [18]:
df['tax_class2'].value_counts() # we just care about the category 1: residential

1    100616
2      4103
3       348
4       126
0        11
Name: tax_class2, dtype: int64

In [19]:
df = df[df.tax_class2 == '1']
# here is where I filter only for ones (residential tax class)

In [20]:
df['tax_class2'].value_counts()

1    100616
Name: tax_class2, dtype: int64

In [21]:
df.homestead.value_counts()

** Currently receiving the Homestead Deduction*.                     49540
** Not receiving the Homestead Deduction                             45475
** Currently receiving the Senior Citizen Homestead Deduction*.       5601
Name: homestead, dtype: int64

In [22]:
homestead1 = pd.get_dummies(df.homestead).iloc[:, :]
homestead1.columns

Index(['** Currently receiving the Homestead Deduction*.  ',
       '** Currently receiving the Senior Citizen Homestead Deduction*.  ',
       '** Not receiving the Homestead Deduction '],
      dtype='object')

In [23]:
homestead1.columns = ['homestead_yes', 'homestead_senior', 'homestead_no']
df = pd.concat([df, homestead1], axis=1)

In [24]:
# consider dropping one of the homestead categories, or all three. for arima it's not necessary

In [25]:
df['assessor'] = df['assessor'].str.title()

In [26]:
df['land_area'] = df['land_area'].str.replace(',', '')

In [27]:
## consider dropping land_area < '4'

In [28]:
df['owner_name'] = df['owner_name'].str.title()

In [29]:
df['sale_price'] = df['sale_price'].str.replace(',', '').str.replace('$', '')

In [30]:
df.drop(['current_value','new_value'], axis=1, inplace=True) # we don't need these

In [31]:
df['land_2017']= df['land_2017'].str.replace(',', '').str.replace('$', '')

In [32]:
df['land_2018']= df['land_2018'].str.replace(',', '').str.replace('$', '')

In [33]:
df['improvements_2017']= df['improvements_2017'].str.replace(',', '').str.replace('$', '')

In [34]:
df['improvements_2018']= df['improvements_2018'].str.replace(',', '').str.replace('$', '')

In [35]:
df['value_2017']= df['value_2017'].str.replace(',', '').str.replace('$', '')

In [36]:
df['value_2018']= df['value_2018'].str.replace(',', '').str.replace('$', '')

In [37]:
df['assessment_2017']= df['assessment_2017'].str.replace(',', '').str.replace('$', '')

In [38]:
df['assessment_2018']= df['assessment_2018'].str.replace(',', '').str.replace('$', '')

In [39]:
# the address column is the mailing address, not the destination. UGHHH

In [40]:
df['address'] = df['address'].str.replace('                  ', ', ')

In [41]:
df['address'] = df['address'].str.replace('                 ', ', ')

In [42]:
df['address'] = df['address'].str.replace('    ', ' ')

In [43]:
df['address'] = df['address'].str.replace('   ', ' ')

In [44]:
df['address'] = df['address'].str.replace('  ', ' ')

In [45]:
df['address'] = df['address'].str.strip()

In [46]:
df.columns

Index(['neighborhood', 'sub_neighborhood', 'exception', 'tax_class',
       'homestead', 'assessor', 'building_area', 'ward', 'land_area',
       'triennial_group', 'owner_name', 'address', 'sale_price', 'date',
       'instrument_no', 'sales_code', 'sales_type', 'land_2017', 'land_2018',
       'improvements_2017', 'improvements_2018', 'value_2017', 'value_2018',
       'assessment_2017', 'assessment_2018', 'use_code2', 'tax_type2',
       'tax_class2', 'homestead_yes', 'homestead_senior', 'homestead_no'],
      dtype='object')

In [47]:
df['address1'] = df['address'] #this gives me something to work with

In [48]:
df['address1'] = df['address1'].str.strip() #trimming 

In [49]:
df['address1'] = df['address1'].str.replace(',', '')

In [50]:
df['address1'] = df['address1'].str.replace('-', ' ')

In [51]:
df['address1'] = df['address1'].str.replace(';', '')

In [52]:
df['address1'] = df['address1'].str.replace('  ', ' ')

In [53]:
df['address1'] = df['address1'].str.replace(u'\xa0', u' ')

In [54]:
df['zip_code'] = df['address1'].str.extract('(\d{5,})', expand = True)

In [55]:
df['state'] = df['address1'].str.extract('(\d{2,}$)', expand = True)

In [56]:
df['address1'] = df['address1'].str.replace('\d{2,}$', '')

In [57]:
df['address1'] = df['address1'].str.strip() #trimming 

In [58]:
df['address1'] = df['address1'].str.replace('\d+$', '') 

In [59]:
df['state'] = df['address1'].str.extract('(\S+$)', expand = True)

In [60]:
df['address1'] = df['address1'].str.replace('\S+$', '') 

In [61]:
df['address1'] = df['address1'].str.strip() #trimming 

In [62]:
df['city'] = df['address1'].str.extract('(\S+$)', expand = True)

In [63]:
df['address_1'] = df['address1'].str.replace('(\S+$)', '')

In [64]:
df['address_1'] = df['address_1'].str.strip() #trimming 

In [65]:
#created address_1 column

In [66]:
df.drop(['address1'], 1, inplace = True)

In [67]:
df.head()

,neighborhood,sub_neighborhood,exception,tax_class,homestead,assessor,building_area,ward,land_area,triennial_group,...,use_code2,tax_type2,tax_class2,homestead_yes,homestead_senior,homestead_no,zip_code,state,city,address_1
0,American University Park,C,No,001 - Residential,** Currently receiving the Homestead Deduction...,Parker Norman,,3.0,7500,3.0,...,12,TX,1,1,0,0,20016,DC,WASHINGTON,4933 MASSACHUSETTS AVE NW
1,American University Park,C,No,001 - Residential,** Currently receiving the Homestead Deduction...,Parker Norman,,3.0,7500,3.0,...,12,TX,1,1,0,0,20016,DC,WASHINGTON,4931 MASSACHUSETTS AVE NW
2,American University Park,C,No,001 - Residential,** Currently receiving the Homestead Deduction...,Parker Norman,,3.0,6000,3.0,...,12,TX,1,1,0,0,20016,DC,WASHINGTON,4923 MASSACHUSETTS AVE NW
3,American University Park,C,No,001 - Residential,** Not receiving the Homestead Deduction,Parker Norman,,3.0,6486,3.0,...,12,TX,1,0,0,1,20814,MD,BETHESDA,7200 WISCONSIN AVE STE 903
4,American University Park,C,No,001 - Residential,** Currently receiving the Homestead Deduction...,Parker Norman,,3.0,5296,3.0,...,12,TX,1,1,0,0,20016,DC,WASHINGTON,4410 49TH ST NW


In [68]:
#if I drop the non-dc addresses, I will keep 87% of my dataset

In [69]:
df.drop_duplicates(['date', 'address_1'], inplace = True)

In [70]:
df.shape

(70082, 35)

In [71]:
#this is a really important part. Dropping duplicates. be very careful here

In [72]:
#dropping! the smallest number
df.drop_duplicates(subset=['address', 'date'], inplace = True)

In [73]:
df.shape

(70082, 35)

In [74]:
len(df[df.state=='DC'])/df.shape[0]


0.8800119859593049

In [75]:
df['sales_type'] = df['sales_type'].str.replace(u'\xa0', u' ')
#\xa0 is  non-breaking space in Latin1 (ISO 8859-1). replace with a space

In [76]:
df['sales_type'].replace(' ', np.nan, inplace=True) #I KNOW that there are fucking empty cells. gotta fill them in.

In [77]:
df.sales_type.unique()

array(['I - IMPROVED', nan, 'V - VACANT'], dtype=object)

In [78]:
df['sale_price'].replace('Not Available', np.nan, inplace = True)

In [79]:
df.shape

(70082, 35)

In [80]:
df.dropna(subset=['sale_price'], how='any', inplace = True)

In [81]:
import datetime as dt
import datetime
df['date']= pd.to_datetime(df.date)

In [82]:
df['qtr'] = df.date.dt.quarter
df['month'] = df.date.dt.month
df['year'] = df.date.dt.year

In [83]:
df['owner_name'] = df['owner_name'].str.replace(u'\xa0', u' ')

In [84]:
df['owner_name'] = df['owner_name'].str.strip()

In [85]:
df.shape

(51667, 38)

In [86]:
df.dtypes

neighborhood                 object
sub_neighborhood             object
exception                    object
tax_class                    object
homestead                    object
assessor                     object
building_area                object
ward                        float64
land_area                    object
triennial_group             float64
owner_name                   object
address                      object
sale_price                   object
date                 datetime64[ns]
instrument_no                object
sales_code                   object
sales_type                   object
land_2017                    object
land_2018                    object
improvements_2017            object
improvements_2018            object
value_2017                   object
value_2018                   object
assessment_2017              object
assessment_2018              object
use_code2                    object
tax_type2                    object
tax_class2                  

In [87]:
# df.loc[df.sale_price <= 3, :]

In [88]:
df = df[df.state == 'DC']


In [89]:
df.tax_class2.unique()

array(['1'], dtype=object)

In [90]:
cols = ['sub_neighborhood', 'exception', 'tax_class',
       'homestead', 'building_area',
       'triennial_group', 'address',
       'instrument_no', 'tax_class2']

In [91]:
df.drop(cols, 1, inplace = True) ## very important cell here!

In [92]:
df = df[df.year >= 2000] ## very important cell here!

In [93]:
#sales_type has NaN

In [94]:
df['improvements_2017'].replace('Not Available', np.NaN, inplace = True)
df['improvements_2018'].replace('Not Available', np.NaN, inplace = True)

In [95]:
for column in df[['land_2017','land_2018', 'improvements_2017',
                 'improvements_2018','value_2017', 'value_2018', 'assessment_2017','assessment_2018']]:         
    df[column] = df[column].astype(float) ### need to filter

In [96]:
df.describe()

,ward,land_2017,land_2018,improvements_2017,improvements_2018,value_2017,value_2018,assessment_2017,assessment_2018,homestead_yes,homestead_senior,homestead_no,qtr,month,year
count,45518.000000,4.551800e+04,4.551800e+04,4.519400e+04,4.523800e+04,4.551800e+04,4.551800e+04,4.551800e+04,4.551800e+04,45518.000000,45518.000000,45518.000000,45518.000000,45518.000000,45518.000000
mean,4.234523,2.629016e+05,2.795920e+05,3.579938e+05,3.736010e+05,6.183471e+05,6.508949e+05,5.536582e+05,5.876878e+05,0.629927,0.024715,0.345358,2.568742,6.715299,2010.556901
std,2.102695,3.632780e+05,3.723926e+05,8.655499e+05,9.781181e+05,1.174895e+06,1.286231e+06,1.173838e+06,1.286159e+06,0.482829,0.155258,0.475490,1.070965,3.277123,5.042640
min,1.000000,1.000000e+01,1.000000e+01,1.000000e+02,1.000000e+02,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,0.000000,0.000000,0.000000,1.000000,1.000000,2000.000000
25%,2.000000,1.169250e+05,1.214200e+05,1.809025e+05,1.895750e+05,3.455100e+05,3.632200e+05,2.806970e+05,2.999950e+05,0.000000,0.000000,0.000000,2.000000,4.000000,2006.000000
50%,4.000000,2.183800e+05,2.306500e+05,2.817550e+05,2.917950e+05,5.050150e+05,5.336650e+05,4.352800e+05,4.640700e+05,1.000000,0.000000,0.000000,3.000000,7.000000,2012.000000
75%,6.000000,3.626000e+05,3.871800e+05,4.105800e+05,4.219275e+05,7.331450e+05,7.672975e+05,6.606482e+05,6.964750e+05,1.000000,0.000000,1.000000,3.000000,9.000000,2015.000000
max,8.000000,3.472220e+07,3.541665e+07,8.285049e+07,8.285049e+07,1.105066e+08,1.105066e+08,1.105066e+08,1.105066e+08,1.000000,1.000000,1.000000,4.000000,12.000000,2017.000000


In [97]:
# Set index
df = df.set_index('date')

In [98]:
df.head()

,neighborhood,assessor,ward,land_area,owner_name,sale_price,sales_code,sales_type,land_2017,land_2018,...,homestead_yes,homestead_senior,homestead_no,zip_code,state,city,address_1,qtr,month,year
date,,,,,,,,,,,,,,,,,,,,,
2015-06-29,American University Park,Parker Norman,3.0,7500,Dan I Magder,1073000,MARKET,I - IMPROVED,526200.0,550650.0,...,1,0,0,20016,DC,WASHINGTON,4933 MASSACHUSETTS AVE NW,2,6,2015
2013-07-15,American University Park,Parker Norman,3.0,6000,Thomas J Leonard,880000,MARKET,I - IMPROVED,480570.0,504050.0,...,1,0,0,20016,DC,WASHINGTON,4923 MASSACHUSETTS AVE NW,3,7,2013
2011-12-27,American University Park,Parker Norman,3.0,5296,Jennifer Budoff,817000,MARKET,I - IMPROVED,494910.0,519860.0,...,1,0,0,20016,DC,WASHINGTON,4410 49TH ST NW,4,12,2011
2010-07-07,American University Park,Parker Norman,3.0,6000,Hyesook Chung,880000,MARKET,I - IMPROVED,505860.0,530580.0,...,0,0,1,20016,DC,WASHINGTON,5047 MASSACHUSETTS AVE NW,3,7,2010
2017-11-27,American University Park,Parker Norman,3.0,5017,John W Hanna,1040000,MARKET,I - IMPROVED,490110.0,515150.0,...,1,0,0,20016,DC,WASHINGTON,4918 ALBEMARLE ST NW,4,11,2017


In [99]:
df.to_csv('otr_clean.csv', encoding='utf-8')

In [100]:
df1 = pd.read_csv('otr_clean.csv', index_col='date')

In [101]:
df1

,neighborhood,assessor,ward,land_area,owner_name,sale_price,sales_code,sales_type,land_2017,land_2018,...,homestead_yes,homestead_senior,homestead_no,zip_code,state,city,address_1,qtr,month,year
date,,,,,,,,,,,,,,,,,,,,,
2015-06-29,American University Park,Parker Norman,3.0,7500,Dan I Magder,1073000,MARKET,I - IMPROVED,526200.0,550650.0,...,1,0,0,20016.0,DC,WASHINGTON,4933 MASSACHUSETTS AVE NW,2,6,2015
2013-07-15,American University Park,Parker Norman,3.0,6000,Thomas J Leonard,880000,MARKET,I - IMPROVED,480570.0,504050.0,...,1,0,0,20016.0,DC,WASHINGTON,4923 MASSACHUSETTS AVE NW,3,7,2013
2011-12-27,American University Park,Parker Norman,3.0,5296,Jennifer Budoff,817000,MARKET,I - IMPROVED,494910.0,519860.0,...,1,0,0,20016.0,DC,WASHINGTON,4410 49TH ST NW,4,12,2011
2010-07-07,American University Park,Parker Norman,3.0,6000,Hyesook Chung,880000,MARKET,I - IMPROVED,505860.0,530580.0,...,0,0,1,20016.0,DC,WASHINGTON,5047 MASSACHUSETTS AVE NW,3,7,2010
2017-11-27,American University Park,Parker Norman,3.0,5017,John W Hanna,1040000,MARKET,I - IMPROVED,490110.0,515150.0,...,1,0,0,20016.0,DC,WASHINGTON,4918 ALBEMARLE ST NW,4,11,2017
2011-12-05,American University Park,Parker Norman,3.0,6099,Matthew F Dunn,870000,MARKET,I - IMPROVED,507440.0,532200.0,...,1,0,0,20016.0,DC,WASHINGTON,4922 ALBEMARLE ST NW,4,12,2011
2005-03-16,American University Park,Parker Norman,3.0,6000,Gerard Janco,840000,MARKET,I - IMPROVED,480570.0,504050.0,...,1,0,0,20016.0,DC,WASHINGTON,4927 MASSACHUSETTS AVE NW,1,3,2005
2003-11-12,American University Park,Parker Norman,3.0,5724,Julie A Mack,575000,MARKET,I - IMPROVED,501710.0,526550.0,...,1,0,0,20016.0,DC,WASHINGTON,4416 49TH ST NW,4,11,2003
2017-04-05,American University Park,Parker Norman,3.0,8006,Garrett C Marquis,989000,MARKET,I - IMPROVED,532400.0,556660.0,...,1,0,0,20016.0,DC,WASHINGTON,4910 ALBEMARLE ST NW,2,4,2017
